In [1]:
import gc
import re
import csv
import nltk
import pickle
import pandas as pd
import contractions
import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelBinarizer

In [2]:
nltk.download('punkt')
nltk.download('words')
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/azureuser/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
!unzip "dataset52a7b21.zip"

Archive:  dataset52a7b21.zip
   creating: dataset/
  inflating: dataset/train.csv       
  inflating: dataset/test.csv        


In [2]:
data = pd.read_csv("dataset/train.csv", escapechar = "\\", quoting = csv.QUOTE_NONE, error_bad_lines=False, na_filter=False)

In [156]:
data1.to_csv('data_128.csv')

In [180]:
data = pd.read_csv("data5.csv", na_filter=False)

In [6]:
data = data.drop_duplicates()

In [9]:
from collections import Counter

In [18]:
labels = Counter(data['BROWSE_NODE_ID'])

In [27]:
print(len(labels))

9919


In [23]:
c, d = 0, 0
for l in labels.keys():
    if(labels[l]<100):
        c += 1
    else:
        d += 1
print(c, d)

7398 2521


In [28]:
remove = []
for i in range(2801530):
    if(labels[data.iloc[i]['BROWSE_NODE_ID']]<100):
        remove.append(i)
print(len(remove))
data_mod = data.drop(data.index[remove])

164476


In [33]:
labels_mod = Counter(data_mod['BROWSE_NODE_ID'])

In [5]:
words = set(nltk.corpus.words.words())

In [6]:
def preprocess_txt(df):
    df = df.apply(unidecode.unidecode)
    df = df.apply(contractions.fix)
    df = df.str.lower()
    df = df.str.replace(r"[^a-z]", " ")
    df = df.apply(word_tokenize)
    df = df.apply(lambda x: [item for item in x if (item in words and len(item)>1) or len(item)>3])
    return df

In [38]:
def preprocess_brand(df):
    df = df.apply(unidecode.unidecode)
    df = df.str.replace(r"[^A-Za-z0-9]", "")
    df = df.str.lower()
    return df

In [48]:
def preprocess_bullet(df):
    df = df.apply(unidecode.unidecode)
    df = df.str.lower()
    df = df.apply(contractions.fix)
    df = df.str.replace(r"[^a-z]", " ")
    df = df.apply(word_tokenize)
    df = df.apply(lambda x: [item for item in x if len(item)>2 and item not in stop])
    return df

In [49]:
data_mod['BULLET_POINTS'] = preprocess_bullet(data_mod['BULLET_POINTS'])

In [ ]:
data_new['Text'] = data['TITLE'] + data['DESCRIPTION']

In [12]:
data_new['Text'] = preprocess_txt(data_new['Text'])

In [ ]:
data_new['BRAND'] = preprocess_brand(data_new['BRAND'])

In [102]:
print(data_new.iloc[1]['Text'])

['the', 'new', 'yorker', 'nyhm', 'refrigerator', 'magnet', 'the', 'new', 'yorker', 'handsome', 'cello', 'wrapped', 'hard', 'magnet', 'measures', 'inch', 'width', 'by', 'inch', 'height', 'and', 'highlight', 'one', 'of', 'the', 'many', 'beautiful', 'new', 'yorker', 'covers', 'in', 'full', 'color', 'cat', 'in', 'tea', 'cup', 'by', 'new', 'yorker', 'cover', 'artist', 'gurbuz', 'dogan', 'eksioglu', 'ideal', 'home', 'office', 'gift', 'magazine', 'lover', 'rigid', 'thenewyorker']


In [ ]:
def processBP(pt):
    pt = re.sub(r"[^a-z]", " ", pt)
    return pt.split()

Merge Bullet Points to Text Field

In [ ]:
for i in range(2637054):
    temp = set(data_new2.iloc[i]['Text'])
    pts = processBP(data_new2.iloc[i]['BULLET_POINTS'])
    for pt in pts:
        if(pt not in temp):
            data_new2.iloc[i]['Text'].append(pt)

Merge Brand to Text Field

In [98]:
for i in range(2637054):
    temp = set(data_new2.iloc[i]['Text'])
    if(data_new2.iloc[i]['BRAND'] not in temp):
        data_new2.iloc[i]['Text'].append(data_new2.iloc[i]['BRAND'])

In [129]:
seriesObj = data_new3.apply(lambda x : True if len(x['Text']) >128 else False, axis = 1)
numOfRows = len(seriesObj[seriesObj == True].index)
print(numOfRows)

789875


In [161]:
seriesObj = data_new3.apply(lambda x : True if (len(x['Text']) >=200 and len(x['Text']) <= 254) else False, axis = 1)
numOfRows = len(seriesObj[seriesObj == True].index)
print(numOfRows)

174914


In [ ]:
seriesObj = data_new3.apply(lambda x : True if (len(x['Text']) >=100 and len(x['Text']) <= 126) else False, axis = 1)
numOfRows = len(seriesObj[seriesObj == True].index)
print(numOfRows)

In [151]:
data1 = data_new3.loc[data_new3.index[seriesObj[seriesObj == True].index.values]]

In [158]:
counter = Counter(data1['BROWSE_NODE_ID'])

In [159]:
print(len(counter))

2513


In [160]:
count = 0
for i in counter.keys():
    if(counter[i]>=100):
        count += 1
print(count)

467


In [162]:
data1.shape

(248974, 2)

In [164]:
remove = []
for i in range(248974):
    if(counter[data1.iloc[i]['BROWSE_NODE_ID']]<100):
        remove.append(i)
print(len(remove))

62441


In [165]:
data_mod = data1.drop(data1.index[remove])

In [166]:
data_mod.shape

(186533, 2)

In [168]:
counter = Counter(data_mod['BROWSE_NODE_ID'])

In [173]:
remove = []
tempdict = {}
for i in range(186533):
    if(data_mod.iloc[i]['BROWSE_NODE_ID'] in tempdict and tempdict[data_mod.iloc[i]['BROWSE_NODE_ID']]<100):
      tempdict[data_mod.iloc[i]['BROWSE_NODE_ID']] += 1
    elif(data_mod.iloc[i]['BROWSE_NODE_ID'] not in tempdict):
      tempdict[data_mod.iloc[i]['BROWSE_NODE_ID']] = 1
    else:
      remove.append(i)

In [174]:
186533- len(remove)

46700

In [175]:
data_mod2 = data_mod.drop(data_mod.index[remove])

In [179]:
data_mod2.shape

(46700, 2)

In [178]:
data_mod2.to_csv('data_467.csv')